In [125]:
import pandas as pd
from workalendar.europe import Russia

In [134]:
df = pd.read_csv('data_agg_weather.csv', sep = ';', index_col=0)

In [135]:
df = df.drop('Unnamed: 0', axis=1)

In [136]:
df

,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб.",День,Средняя Температура,Снег,Дождь
0,2022-09-01,3,9,2022,1,0,3,96630.00,1,11.4,0,0
1,2022-09-02,4,9,2022,1,0,3,108024.50,2,10.5,0,1
2,2022-09-03,5,9,2022,1,0,3,122352.00,3,12.5,0,0
3,2022-09-04,6,9,2022,1,0,3,119441.70,4,12.2,0,0
4,2022-09-05,0,9,2022,1,0,3,87863.50,5,10.3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
415,2023-10-22,6,10,2023,4,0,3,138996.12,22,2.6,1,0
416,2023-10-23,0,10,2023,4,0,3,61260.00,23,4.4,0,1
417,2023-10-24,1,10,2023,4,0,3,70993.00,24,1.4,0,1
418,2023-10-25,2,10,2023,4,0,3,85317.97,25,-0.3,0,0


In [128]:
cal = Russia()

In [129]:
cal.holidays(2023)

[(datetime.date(2023, 1, 1), 'New year'),
 (datetime.date(2023, 1, 2), 'Day After New Year'),
 (datetime.date(2023, 1, 3), 'Third Day after New Year'),
 (datetime.date(2023, 1, 4), 'Fourth Day after New Year'),
 (datetime.date(2023, 1, 5), 'Fifth Day after New Year'),
 (datetime.date(2023, 1, 6), 'Sixth Day after New Year'),
 (datetime.date(2023, 1, 7), 'Christmas'),
 (datetime.date(2023, 1, 8), 'Eighth Day after New Year'),
 (datetime.date(2023, 2, 23), 'Defendence of the Fatherland'),
 (datetime.date(2023, 3, 8), "International Women's Day"),
 (datetime.date(2023, 5, 1), 'The Day of Spring and Labour'),
 (datetime.date(2023, 5, 9), 'Victory Day'),
 (datetime.date(2023, 6, 12), 'National Day'),
 (datetime.date(2023, 11, 4), 'Day of Unity'),
 (datetime.date(2023, 11, 6), 'Day of Unity shift')]

In [107]:
df['Праздник_вых'] = df['Открыт_дата'].apply(lambda x: cal.is_holiday(pd.to_datetime(x)))

День студента

In [108]:
df['Праздник_раб'] = df['Открыт_дата'].apply(lambda x: 1 
                                             if pd.to_datetime(x).month == 1  and pd.to_datetime(x).day == 25
                                            else 0)

День всех влюбленных

In [109]:
df['Праздник_раб'] = df['Открыт_дата'].apply(lambda x: 1 
                                             if pd.to_datetime(x).month == 2  and pd.to_datetime(x).day == 14
                                            else 0)

День знаний

In [110]:
df['Праздник_раб'] = df['Открыт_дата'].apply(lambda x: 1 
                                             if pd.to_datetime(x).month == 9  and pd.to_datetime(x).day == 1
                                            else 0)

День учителя

In [111]:
df['Праздник_раб'] = df['Открыт_дата'].apply(lambda x: 1 
                                             if pd.to_datetime(x).month == 10  and pd.to_datetime(x).day == 5
                                            else 0)

Пасха

In [112]:
def easter_date(god):
    a = god % 19
    b = god % 4
    c = god % 7
    d = (19 * a + 15) % 30
    e = (2 * b + 4 * c + 6 * d + 6) % 7
    f = d + e
    if f <= 26:
        rez = str(f + 4) + '.04'
    else:
        rez = str(f - 26) + '.05'
    return rez + '-' + str(god)

In [113]:
df['Праздник_раб'] = df['Открыт_дата'].apply(lambda x: 1 
                                             if easter_date(pd.to_datetime(x).year) == 
                                             (str(pd.to_datetime(x).month) + '-' + str(pd.to_datetime(x).day))
                                            else 0)

In [114]:
df['Праздники'] = df['Праздник_вых'].astype(int) + df['Праздник_раб']

In [115]:
df['Предпраздничный'] = df['Праздники'].shift(-1)
mask = df['Предпраздничный'] == df['Праздники']

# Проставляем ноль во второй колонке там, где значения не равны
df.loc[mask, 'Предпраздничный'] = 0

# Заменяем NaN в последней строке значением False
df.at[df.index[-1], 'Предпраздничный'] = 0

In [118]:
df.to_csv('data_agg_final.csv', index=False)